# En este Notebook ejecutaremos codificación ordinal, one-hot y codificación por frecuencia y porcentajes, informada por el target y otras técnicas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Para la codificacion ordinal y one-hot usando sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# Para dividir el conjunto de datos
from sklearn.model_selection import train_test_split

In [ ]:
# Cargando datos
data = pd.read_csv('../../datasets/raw/credict_approval.csv')
data.head()

### Codificación ordinal utilizando sklearn

In [ ]:
# Hacemos una lista con las variables categóricas

vars_categorical = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [ ]:
# Separamos los datos en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['A16'], axis=1),  # predictores
    data['A16'],  # target
    test_size=0.3,  
    random_state=0)  # seed para reproducibilidad

X_train.shape, X_test.shape

In [ ]:
# Instanciamos y entrenamos el codificador entero
le = OrdinalEncoder()
le.fit(X_train[vars_categorical])

le.categories_

In [ ]:
# Transformamos los sets de datos y visualizamos

X_train_enc = le.transform(X_train[vars_categorical])
X_test_enc = le.transform(X_test[vars_categorical])

pd.DataFrame(X_train_enc, columns=vars_categorical).head()

### Codificacion one-hot utilizando sklearn

In [ ]:
# Instanciamos el y entrenamos codificador

encoder = OneHotEncoder(categories='auto',
                        drop='first',  
                        sparse_output=False)

encoder.fit(X_train[vars_categorical])

In [ ]:
# Observamos las categorías aprendidas

encoder.categories_

In [ ]:
# Transformamos e inspeccionamos los sets de datos

X_train_enc = encoder.transform(X_train[vars_categorical])
X_test_enc = encoder.transform(X_test[vars_categorical])

pd.DataFrame(X_train_enc).head()
#pd.DataFrame(X_test_enc).head()

In [ ]:
# Podemos recuperar los nombres de las variables de la siguiente manera:

encoder.get_feature_names_out()

In [ ]:
# Visualizamos los valores más frecuentes de A7

X_train["A7"].value_counts().sort_values(ascending=False)

Codificación One hot categorías mas frecuentes


In [ ]:
ohe_enc = OneHotEncoder(
    handle_unknown="infrequent_if_exist",  # categorías no vistas se tratarán como las menos frecuentes
    max_categories=5,  # el número máximo de categorías
    sparse_output=False,  
)

ohe_enc.set_output(transform="pandas")

ohe_enc.fit(X_train)

In [ ]:
ohe_enc.infrequent_categories_

In [ ]:
# codificacación
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

# exploremos los resultados
X_train.head()

### Codificacion por frecuencia y porcentajes

In [ ]:
# Separamos los datos en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data,  # datos completos
    data['A16'],  # target
    test_size=0.3,  
    random_state=0)  # seed para reproducibilidad

X_train.shape, X_test.shape

In [ ]:
#Creamos funciones de procesamiento
def count_mappings(df, variable):
    return df[variable].value_counts().to_dict()


def frequency_mappings(df, variable):
    return (df[variable].value_counts() / len(df)).to_dict()


def encode(train, test, variable, mapping):
    train[variable] = train[variable].map(mapping)
    test[variable] = test[variable].map(mapping)

In [ ]:
# Codificamos múltiples variables aplicando las funciones
for variable in vars_categorical:
    mappings = count_mappings(X_train, variable)
    encode(X_train, X_test, variable, mappings)


X_train.head()


### Codificación informada por la media

In [ ]:
# Separamos los datos en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data,  # datos completos
    data['A16'],  # target
    test_size=0.3,  
    random_state=0)  # seed para reproducibilidad

X_train.shape, X_test.shape

In [ ]:
X_train.groupby(['A7'])['A16'].mean().plot()
plt.title('Relación entre A7 y el target')
plt.ylabel('Media del target')
plt.show()

In [ ]:
# Creamos un diccionario con la media del target por categoría para la variable A7

ordered_labels = X_train.groupby(['A7'])['A16'].mean().to_dict()

ordered_labels

In [ ]:
# Reemplazamos las etiquetas con la media del target e inspeccionamos resultados

X_train['A7'] = X_train['A7'].map(ordered_labels)
X_test['A7'] = X_test['A7'].map(ordered_labels)

X_train['A7'].head()

In [ ]:
X_train.groupby(['A7'])['A16'].mean().plot()
plt.title('Relación entre A7 y el target')
plt.ylabel('Media del target')
plt.show()

### Codificación binaria

Se deja como ejercicio

### Codificación polar

Se deja como ejercicio